## 朴素贝叶斯
> 在机器学习中，朴素贝叶斯分类器是一系列以假设特征之间强（朴素）独立下运用贝叶斯定理为基础的简单概率分类器。
朴素贝叶斯自20世纪50年代已广泛研究。在20世纪60年代初就以另外一个名称引入到文本信息检索界中，并仍然是文本分类的一种热门（基准）方法，文本分类是以词频为特征判断文件所属类别或其他（如垃圾邮件、合法性、体育或政治等等）的问题。通过适当的预处理，它可以与这个领域更先进的方法（包括支持向量机）相竞争。它在自动医疗诊断中也有应用。

>朴素贝叶斯分类器是高度可扩展的，因此需要数量与学习问题中的变量（特征/预测器）成线性关系的参数。最大似然训练可以通过评估一个封闭形式的表达式来完成，只需花费线性时间，而不需要其他很多类型的分类器所使用的费时的迭代逼近。

>在统计学和计算机科学文献中，朴素贝叶斯模型有各种名称，包括简单贝叶斯和独立贝叶斯。所有这些名称都参考了贝叶斯定理在该分类器的决策规则中的使用，但朴素贝叶斯不（一定）用到贝叶斯方法；《Russell和Norvig》提到“‘朴素贝叶斯’有时被称为贝叶斯分类器，这个马虎的使用促使真正的贝叶斯论者称之为傻瓜贝叶斯模型。”----摘自[维基百科](https://zh.wikipedia.org/wiki/%E6%9C%B4%E7%B4%A0%E8%B4%9D%E5%8F%B6%E6%96%AF%E5%88%86%E7%B1%BB%E5%99%A8)

>朴素贝叶斯是一种构建分类器的简单方法。该分类器模型会给问题实例分配用特征值表示的类标签，类标签取自有限集合。它不是训练这种分类器的单一算法，而是一系列基于相同原理的算法：所有朴素贝叶斯分类器都假定样本每个特征与其他特征都不相关。举个例子，如果一种水果其具有红，圆，直径大概3英寸等特征，该水果可以被判定为是苹果。尽管这些特征相互依赖或者有些特征由其他特征决定，然而朴素贝叶斯分类器认为这些属性在判定该水果是否为苹果的概率分布上独立的。

>对于某些类型的概率模型，在监督式学习的样本集中能获取得非常好的分类效果。在许多实际应用中，朴素贝叶斯模型参数估计使用最大似然估计方法；换而言之，在不用到贝叶斯概率或者任何贝叶斯模型的情况下，朴素贝叶斯模型也能奏效。

### 1. 收集数据
以在线社区的留言板为例，如果某条留言使用了负面或者侮辱性的语言，那么就将该留言标识为内容不当，用1表示；正常留言用0表示。

### 2. 准备数据：从文本中构建词向量

In [22]:
def load_data_set():
    """创建实验样本，假设这六个列表为六篇文章"""
    posting_list = [
        ['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
        ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
        ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
        ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
        ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
        ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']
    ]
    class_vec = [0, 1, 0, 1, 0, 1] # 1代表含有侮辱性文字的言论；0代表正常的言论
    return posting_list, class_vec

In [23]:
def create_vocab_list(data_set):
    """创建一个包含在所有文档中不重复出现的词的列表"""
    vocab_set = set([])
    for document in data_set:
        vocab_set = vocab_set | set(document)  # 创建并集
    return list(vocab_set)

In [24]:
def words_set2vec(vocab_list, input_set):
    """将一篇文档中所有不重复的词转换为0-1向量"""
    return_vec = [0] * len(vocab_list)
    for word in input_set:
        if word in vocab_list:
            return_vec[vocab_list.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return return_vec

In [25]:
posting_list, class_vec = load_data_set()
posting_list

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

In [27]:
my_vocab_list = create_vocab_list(posting_list)
print(my_vocab_list) # 可以看到此表中没有重复的词

['stupid', 'to', 'licks', 'problems', 'him', 'dog', 'please', 'steak', 'park', 'garbage', 'ate', 'posting', 'how', 'flea', 'so', 'I', 'dalmation', 'is', 'quit', 'not', 'buying', 'help', 'my', 'love', 'has', 'take', 'food', 'mr', 'worthless', 'maybe', 'stop', 'cute']


In [28]:
"""假设将第一(索引为0)篇文章里的所有词当作侮辱性词，
    检查my_vocab_list中每个词的词性（是否含侮辱）"""
my_vec = words_set2vec(my_vocab_list, posting_list[0]) 
print(my_vec)

[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]


可以看到my_vocab_list中的第四个词‘problems’是在第一篇文章中出现，所以被标记为1